## Acquire raw temperature dataset from a local server PostgreSQL table using psycopg2

table name = "temperature_sensor_recording"

In [ ]:
import psycopg2
import pandas as pd

# Database connection parameters
db_params = {
    "dbname": "postgres", # use your actual database name (defaut = "postgres")
    "user": "username", # use your actual database username (defaut = "postgres")
    "password": "********", # use your actual database password
    "host": "localhost",  # for local server database
    "port": "5432" # use your actual database port (defaut = "5432")
}

try:
    # Establish connection to PostgreSQL database
    conn = psycopg2.connect(**db_params)
    
    # Query the entire temperature_sensor_recording table
    query = "SELECT * FROM temperature_sensor_recording"
    
    # Read the query results directly into a pandas DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Close the connection
    conn.close()
    
    # Display basic information about the DataFrame
    print("Data loaded successfully!")
    print(f"\nShape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    print(df.head())
    
except psycopg2.Error as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"Error: {e}")

## Feature Extraction

Indicators:
1. start_time: starting hour of the row
2. availability_status: a flag that marks wheather there are 3 or more temperature data in an hour 

Features:
1. num_data: number of temperature data in an hour
2. sampling_rate: number of temperature data in a minute
3. range: difference between maximum and minimum recorded temp data in an hour
4. standard_deviation: standard deviation of recorded temp data in an hour
5. covar:  standard deviation/mean of recorded temp data in an hour
6. max_rate: maximum absolute temperature change rate in a minute
7. min_rate: minimum absolute temperature change rate in a minute
8. rate_range: max_rate - min_rate
9. osc_freq: the number of temperature rate direction change in a minute
10. var_score: percentage of anomalous temperature change in an hour
11. hour_index: index of start_time hour in UTC+7 

In [ ]:
# EDITED

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

df = raw_data

logger.info(f"Loaded {len(df)} rows from CSV")

# Convert time_received to datetime and round down to minute
logger.info("Processing timestamps...")
df['time_received'] = pd.to_datetime(df['time_received'])
df['start_time'] = df['time_received'].dt.floor('T')  # Round down to minute

# Use temp_n as temperature
temp_n = "temp_1" # choose the "temp_1" raw data column instead of the "temp_2"
df['temp'] = df[temp_n] # create another column that consists of the actual processed "temp_1" data

# Remove rows with null temperature
df = df[df['temp'].notna()].copy()   # delete rows with invalid NaN values 
logger.info(f"After removing null temperatures: {len(df)} rows") # print current valid rows

# Create hour_start column (truncate to hour)
df['hour_start'] = df['start_time'].dt.floor('H')  # create another column "hour_start" consisting of the hour-rounded "start_time" 

# Get date range
min_date = df['start_time'].min().date()   # first date in the dataset
max_date = df['start_time'].max().date()   # last date in the dataset
logger.info(f"Data range: {min_date} to {max_date}")

# Generate all dates and hours
logger.info("Generating complete date-hour grid...")
all_results = []  # create an empty list "all_results" to store the extracted features (in dictionaries)

current_date = datetime.combine(min_date, datetime.min.time())  # create starting date as min_date plus time (00:00:00) as the very beginning of the date
end_date = datetime.combine(max_date, datetime.min.time()) + timedelta(days=1)  # create the farthest datetime (max_date + 1, 00:00:00)

total_days = (end_date - current_date).days   # calculate the total number of days in the dataset
processed_days = 0

while current_date < end_date:
    # Process all 24 hours for this date
    for hour in range(24):  # from 0 to 23
        hour_start = current_date + timedelta(hours=hour) # iterate through hours of the current_date 
        
        # Filter data for this hour
        hour_data = df[df['hour_start'] == hour_start].copy() # take all temperature recording rows within this hour as a new df (hour_data)
        num_data = len(hour_data) # define num_data as the number of data rows in an hour

        if num_data < 3:   # if there are less than 3 temperature reecordings within this hour:
            # fill every feature with 0
            all_results.append({
                'num_data': 0,
                'sampling_rate': 0.0,
                'start_time': hour_start,
                'availability_status': 0,
                'range': 0.0,
                'standard_deviation': 0.0,
                'covar': 0.0,
                'max_rate': 0.0,
                'rate_range': 0.0,
                'osc_freq': 0.0,
                'var_score': 0.0,
                'hour_index': (hour_start.hour + 7) % 24
            })
        
        
        else:
            availability_status = 1 # set availability_status to 1 if num_data >=3
            
            # Sort by time
            hour_data = hour_data.sort_values('start_time') # re-arrange hour_data rows based on "start_time" values
            temps = hour_data['temp'].values  # create numpy array of temperature readings
            times = hour_data['start_time'].values # create numpy array of start times
            
            # Basic statistics
            temp_range = float(np.max(temps) - np.min(temps)) # find the difference between max and min temperatures within the hour
            std_dev = float(np.std(temps, ddof=1)) # hourly temperature standard deviation 
            mean_temp = float(np.mean(temps)) # average temp data in this hour
            covar = (std_dev / abs(mean_temp)) if mean_temp != 0 else 0.0 # coefficient of variation
            
            
            rates = []
            abnormal_count = 0
            
            for i in range(len(temps) - 1):
                temp_diff = abs(temps[i+1] - temps[i])
                time_diff_minutes = (times[i+1] - times[i]) / np.timedelta64(1, 'm')
                if time_diff_minutes > 0:
                    rate = temp_diff / time_diff_minutes  # absolute temperature change rate in a minute
                    rates.append(rate) # store temperature rates (change per minute)

                    threshold = 0.031 # celsius/minute
                    if rate > threshold:
                        abnormal_count += 1
            
            max_rate = float(max(rates)) if rates else 0.0 # maximum absolute temperature change rate in a minute
            min_rate = float(min(rates)) if rates else 0.0 # minimum absolute temperature change rate in a minute
            rate_range = max_rate - min_rate # largest temperature change rate difference in an hour 
            
            var_score = float(abnormal_count / len(rates)) if rates else 0.0 # calculate var_score (variability score), Threshold: 0.031°C/minute
            
            # Calculate oscillation frequency
            directions = []
            for i in range(len(temps) - 1):
                if temps[i+1] > temps[i]:
                    directions.append(1)
                elif temps[i+1] < temps[i]:
                    directions.append(-1)
            
            direction_changes = 0
            for i in range(len(directions) - 1):
                if directions[i] != directions[i+1]:
                    direction_changes += 1
            
            osc_freq = float((1 + direction_changes) / 60.0) if directions else 0.0
            
            # Replace any NaN or inf with 0
            std_dev = 0.0 if np.isnan(std_dev) or np.isinf(std_dev) else std_dev
            covar = 0.0 if np.isnan(covar) or np.isinf(covar) else covar
            max_rate = 0.0 if np.isnan(max_rate) or np.isinf(max_rate) else max_rate
            rate_range = 0.0 if np.isnan(rate_range) or np.isinf(rate_range) else rate_range
            var_score = 0.0 if np.isnan(var_score) or np.isinf(var_score) else var_score
            osc_freq = 0.0 if np.isnan(osc_freq) or np.isinf(osc_freq) else osc_freq
            
            all_results.append({
                'num_data': num_data,
                'sampling_rate': num_data / 60.0,
                'start_time': hour_start,
                'availability_status': availability_status,
                'range': temp_range,
                'standard_deviation': std_dev,
                'covar': covar,
                'max_rate': max_rate,
                'rate_range': rate_range,
                'osc_freq': osc_freq,
                'var_score': var_score,
                'hour_index': (hour_start.hour + 7) % 24
            })
    
    # Progress logging
    processed_days += 1
    if processed_days % 10 == 0:
        logger.info(f"Progress: {processed_days}/{total_days} days processed")
    
    # Move to next date
    current_date += timedelta(days=1)

logger.info(f"Total days processed: {processed_days}")

# Convert to DataFrame
logger.info("Creating output DataFrame...")
result_df = pd.DataFrame(all_results)

# Ensure all columns are present and in correct order
output_columns = [
    'start_time', 'hour_index', 'num_data', 'sampling_rate', 'availability_status',
    'range', 'standard_deviation', 'covar', 'max_rate', 'rate_range',
    'osc_freq', 'var_score' 
]

result_df = result_df[output_columns]

# Replace any remaining NaN with 0
result_df = result_df.fillna(0)


# Show statistics
logger.info("=" * 60)
logger.info("PROCESSING COMPLETED!")
logger.info(f"Total hourly records: {len(result_df)}")
logger.info(f"Records with availability_status=1: {result_df['availability_status'].sum()}")
logger.info(f"Records with availability_status=0: {len(result_df) - result_df['availability_status'].sum()}")
logger.info(f"Average sampling rate: {result_df['sampling_rate'].mean():.3f}")
logger.info(f"Average var_score: {result_df['var_score'].mean():.3f}")
logger.info("=" * 60)

# Show sample results
logger.info("\nValid rows:")
df_available = result_df[result_df['availability_status'] == 1].copy()
df_available

## Feature correlation matrix plot

plot the correlation coefficient between each feature pairs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Filter for availability_status == 1
logger.info(f"Records with availability_status=1: {len(df_available)}")
logger.info(f"Percentage available: {len(df_available)/len(result_df)*100:.2f}%")

# Define features for correlation analysis
feature_columns = [
    "num_data", "sampling_rate", "range", "standard_deviation", 
    "covar", "max_rate", "rate_range", "osc_freq", "var_score", "hour_index"
]

# Extract feature subset
X = df_available[feature_columns].copy()


# Create correlation matrix
correlation_matrix = X.corr()

# Create the plot
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True,  # Show correlation values
            cmap='coolwarm',  # Color scheme
            center=0,  # Center colormap at 0
            fmt='.2f',  # Format numbers to 2 decimal places
            square=True,  # Make cells square-shaped
            linewidths=0.5,  # Add gridlines
            cbar_kws={"shrink": 0.8})  # Adjust colorbar size

plt.title('Correlation Matrix of Features', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Optional: Print strongest correlations
logger.info("\nStrongest correlations (absolute value > 0.5):")
correlation_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:
            correlation_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                corr_value
            ))

for feat1, feat2, corr in sorted(correlation_pairs, key=lambda x: abs(x[2]), reverse=True):
    logger.info(f"{feat1} <-> {feat2}: {corr:.3f}")

## Isolation Forest outlier detection
unique features with low correlation coefficients: sampling_rate, rate_range, var_score, hour_index

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

unique_features = ["sampling_rate", "rate_range", "var_score", "hour_index"]
df_features = df_available[unique_features].copy() # df with only available_status == 1, with unique feature columns

scaler = StandardScaler()
df_features_scaled = scaler.fit_transform(df_features)

# Initialize Isolation Forest
model = IsolationForest(
    n_estimators=400,
    max_samples='auto',
    contamination=0.1,  # Expect 10% outliers (adjust based on your domain knowledge)
    random_state=42,  # For reproducibility
    n_jobs=-1  # Use all CPU cores for faster training
)

# Fit the model
model.fit(df_features_scaled)

# Get predictions
# -1 = outlier, 1 = inlier
prediction_labels = model.predict(df_features_scaled)
pred_labels = []

for i in range (len (prediction_labels)):
    if prediction_labels[i] == -1:
        pred_labels.append(0)
    else:
        pred_labels.append(1)
        
# Get anomaly scores (more negative = more anomalous)
anomaly_scores = model.decision_function(df_features_scaled)

min_score = anomaly_scores.min()
max_score = anomaly_scores.max()

normalized_scores = (anomaly_scores - min_score) / (max_score - min_score) # normalize anomaly_scores

anomaly_scores = normalized_scores.copy()

# Add to original dataframe
df_available["pred_labels"] = pred_labels
df_available["anomaly_score"] = anomaly_scores

# Results
df_available

## Hourly temperature sensor diagnosis finalization

creating a complete hourly diagnosis result dataframe (df_hourly) that contains the extracted features, anomaly score, and hourly health status based on the isolation forest outlier detection results

In [ ]:
import pandas as pd
import numpy as np
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info("Starting merge process...")
logger.info(f"df shape: {result_df.shape}")
logger.info(f"df_available shape: {df_available.shape}")

# Create a copy of df to avoid modifying original
df_hourly = result_df.copy()

# Verify that df_available is a subset of df
n_available = (df_hourly['availability_status'] == 1).sum()
logger.info(f"Rows with availability_status == 1 in df_hourly: {n_available}")
logger.info(f"Rows in df_available: {len(df_available)}")

if n_available != len(df_available):
    logger.warning(f"Mismatch detected! Expected {n_available} but got {len(df_available)}")

# Step 1: Prepare df_available for merging
# Select only the columns we need and rename pred_labels to health_status
df_available_merge = df_available[['start_time', 'pred_labels', 'anomaly_score']].copy()
df_available_merge = df_available_merge.rename(columns={'pred_labels': 'health_status'})

logger.info("\nPrepared df_available for merging:")
logger.info(f"Columns: {df_available_merge.columns.tolist()}")
logger.info(f"health_status value counts:\n{df_available_merge['health_status'].value_counts()}")

# Step 2: Merge df_hourly with df_available on start_time (left join)
logger.info("\nPerforming left merge on 'start_time'...")
df_hourly = df_hourly.merge(
    df_available_merge,
    on='start_time',
    how='left',
    suffixes=('', '_from_available')
)

logger.info(f"After merge, df_hourly shape: {df_hourly.shape}")

# Step 3: Fill NaN values based on availability_status
logger.info("\nFilling NaN values based on availability_status...")

# For rows where availability_status == 0, fill with defaults
mask_unavailable = df_hourly['availability_status'] == 0
n_unavailable = mask_unavailable.sum()

df_hourly.loc[mask_unavailable, 'health_status'] = 1
df_hourly.loc[mask_unavailable, 'anomaly_score'] = 0.0

logger.info(f"Filled {n_unavailable} rows with availability_status == 0")
logger.info(f"  health_status = 1 (healthy)")
logger.info(f"  anomaly_score = 0.0")

# Step 4: Verify no NaN values remain in health_status and anomaly_score
nan_health = df_hourly['health_status'].isna().sum()
nan_anomaly = df_hourly['anomaly_score'].isna().sum()

if nan_health > 0 or nan_anomaly > 0:
    logger.warning(f"\nWarning: Found unexpected NaN values!")
    logger.warning(f"  NaN in health_status: {nan_health}")
    logger.warning(f"  NaN in anomaly_score: {nan_anomaly}")
    
    # Show which rows have NaN
    if nan_health > 0:
        logger.warning("\nRows with NaN health_status:")
        print(df_hourly[df_hourly['health_status'].isna()][['start_time', 'availability_status']].head(10))
else:
    logger.info("\n✓ No NaN values found in health_status or anomaly_score")

# Step 5: Ensure correct data types
df_hourly['health_status'] = df_hourly['health_status'].astype(int)
df_hourly['anomaly_score'] = df_hourly['anomaly_score'].astype(float)

# Step 6: Print summary statistics
logger.info("\n" + "="*80)
logger.info("MERGE SUMMARY")
logger.info("="*80)
logger.info(f"\nTotal rows: {len(df_hourly)}")
logger.info(f"\nAvailability Status distribution:")
logger.info(df_hourly['availability_status'].value_counts().to_string())

logger.info(f"\nHealth Status distribution:")
health_counts = df_hourly['health_status'].value_counts().sort_index()
for status, count in health_counts.items():
    pct = count / len(df_hourly) * 100
    status_name = "Anomaly/Unhealthy" if status == 0 else "Normal/Healthy"
    logger.info(f"  {status} ({status_name}): {count} ({pct:.2f}%)")

logger.info(f"\nAnomaly Score statistics:")
logger.info(f"  Min: {df_hourly['anomaly_score'].min():.4f}")
logger.info(f"  Max: {df_hourly['anomaly_score'].max():.4f}")
logger.info(f"  Mean: {df_hourly['anomaly_score'].mean():.4f}")
logger.info(f"  Median: {df_hourly['anomaly_score'].median():.4f}")

# Cross-tabulation: availability_status vs health_status
logger.info(f"\nCross-tabulation (availability_status vs health_status):")
crosstab = pd.crosstab(
    df_hourly['availability_status'], 
    df_hourly['health_status'],
    margins=True,
    margins_name='Total'
)
print(crosstab)

logger.info("\n" + "="*80)
logger.info("MERGE COMPLETED SUCCESSFULLY!")
logger.info("="*80)

## Daily temperature sensor health scoring

calculation of: 
1. transmission health score (transmission_hs): the percentage of availaibility_status in each day
2. sensor health score (sensor_hs): the percentage of health_status in each day

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.info(f"Loaded {len(df_hourly)} hourly records")

# Parse start_time as datetime
logger.info("Parsing start_time as datetime...")
df_hourly['start_time'] = pd.to_datetime(df_hourly['start_time'])

# Extract date from start_time
df_hourly['date'] = df_hourly['start_time'].dt.date

# Get date range
min_date = df_hourly['date'].min()   # oldest date in df_hourly ['date']
max_date = df_hourly['date'].max()   # latest date in df_hourly ['date']
logger.info(f"Date range: {min_date} to {max_date}")

total_days = (max_date - min_date).days + 1
logger.info(f"Total days to process: {total_days}")

# Initialize list to store daily results
daily_results = []

# Process day by day
logger.info("\nProcessing daily summaries...")
current_date = min_date
processed_days = 0

while current_date <= max_date: # iterate througgh dates in df_hourly
    # Filter data for this specific day
    day_data = df_hourly[df_hourly['date'] == current_date] # take only the hourly data in the current_date 
    
    # Count hours in this day
    n_hours = len(day_data)
    
    # Count availability_status == 1
    n_available = (day_data['availability_status'] == 1).sum()
    
    # Count health_status == 1
    n_healthy = (day_data['health_status'] == 1).sum()
    
    # Calculate percentages (always divide by 24)
    transmission_hs = int((n_available / 24) * 100)
    sensor_hs = int((n_healthy / 24) * 100)
    
    # Store result
    daily_results.append({
        'date': current_date.strftime('%Y-%m-%d'),
        'transmission_hs': transmission_hs,
        'sensor_hs': sensor_hs
    })
    
    # Log if day has incomplete data
    if n_hours != 24:
        logger.warning(f"  {current_date}: Only {n_hours}/24 hours found!")
    
    # Progress logging
    processed_days += 1
    if processed_days % 30 == 0:
        logger.info(f"Progress: {processed_days}/{total_days} days processed")
    
    # Move to next day
    current_date += timedelta(days=1)

logger.info(f"Completed processing {processed_days} days")

# Create daily DataFrame
logger.info("\nCreating daily diagnosis DataFrame...")
df_daily = pd.DataFrame(daily_results)

# Ensure correct data types
df_daily['transmission_hs'] = df_daily['transmission_hs'].astype(int)
df_daily['sensor_hs'] = df_daily['sensor_hs'].astype(int)

# Print summary statistics
logger.info("\n" + "="*80)
logger.info("DAILY DIAGNOSIS SUMMARY")
logger.info("="*80)
logger.info(f"\nTotal days: {len(df_daily)}")

logger.info(f"\nTransmission Health Status (transmission_hs):")
logger.info(f"  Min: {df_daily['transmission_hs'].min()}%")
logger.info(f"  Max: {df_daily['transmission_hs'].max()}%")
logger.info(f"  Mean: {df_daily['transmission_hs'].mean():.2f}%")
logger.info(f"  Median: {df_daily['transmission_hs'].median()}%")

logger.info(f"\nSensor Health Status (sensor_hs):")
logger.info(f"  Min: {df_daily['sensor_hs'].min()}%")
logger.info(f"  Max: {df_daily['sensor_hs'].max()}%")
logger.info(f"  Mean: {df_daily['sensor_hs'].mean():.2f}%")
logger.info(f"  Median: {df_daily['sensor_hs'].median()}%")

# Days with perfect transmission
perfect_transmission = (df_daily['transmission_hs'] == 100).sum()
logger.info(f"\nDays with 100% transmission: {perfect_transmission} ({perfect_transmission/len(df_daily)*100:.2f}%)")

# Days with perfect sensor health
perfect_sensor = (df_daily['sensor_hs'] == 100).sum()
logger.info(f"Days with 100% sensor health: {perfect_sensor} ({perfect_sensor/len(df_daily)*100:.2f}%)")

# Days with both perfect
both_perfect = ((df_daily['transmission_hs'] == 100) & (df_daily['sensor_hs'] == 100)).sum()
logger.info(f"Days with both 100%: {both_perfect} ({both_perfect/len(df_daily)*100:.2f}%)")

# Distribution bins
logger.info(f"\nTransmission Health Distribution:")
transmission_bins = pd.cut(df_daily['transmission_hs'], bins=[0, 25, 50, 75, 100], 
                            labels=['0-25%', '26-50%', '51-75%', '76-100%'], 
                            include_lowest=True)
logger.info(transmission_bins.value_counts().sort_index().to_string())

logger.info(f"\nSensor Health Distribution:")
sensor_bins = pd.cut(df_daily['sensor_hs'], bins=[0, 25, 50, 75, 100], 
                     labels=['0-25%', '26-50%', '51-75%', '76-100%'], 
                     include_lowest=True)
logger.info(sensor_bins.value_counts().sort_index().to_string())

# Show sample data
logger.info(f"\nFirst 10 days:")
print(df_daily.head(10).to_string(index=False))

logger.info(f"\nLast 10 days:")
print(df_daily.tail(10).to_string(index=False))

logger.info("\n" + "="*80)
logger.info("DAILY DIAGNOSIS CREATION COMPLETED!")
logger.info("="*80)

## Upload df_hourly and df_daily health assesment results into postgres tables

df_hourly ==> tempsensor_hourly_healthstatus

df_daily ==> tempsensor_daily_healthscore

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
db_params = {
    "dbname": "postgres",
    "user": "Username",
    "password": "******",
    "host": "localhost",  
    "port": "5432"
}

# Assuming you have df_hourly and df_daily DataFrames already created
# df_hourly = your hourly data
# df_daily = your daily data

try:
    # Create SQLAlchemy engine for easier DataFrame upload
    connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
    engine = create_engine(connection_string)
    
    # Upload df_hourly to tempsensor_hourly_healthstatus table
    df_hourly.to_sql(
        name='tempsensor_hourly_healthstatus',
        con=engine,
        if_exists='replace',  # Options: 'fail', 'replace', 'append'
        index=False  # Don't write DataFrame index as a column
    )
    print("✓ df_hourly uploaded to tempsensor_hourly_healthstatus")
    
    # Upload df_daily to tempsensor_daily_healthscore table
    df_daily.to_sql(
        name='tempsensor_daily_healthscore',
        con=engine,
        if_exists='replace',  # Options: 'fail', 'replace', 'append'
        index=False  # Don't write DataFrame index as a column
    )
    print("✓ df_daily uploaded to tempsensor_daily_healthscore")
    
    # Close the engine
    engine.dispose()
    
    print("\nBoth DataFrames uploaded successfully!")
    
except Exception as e:
    print(f"Error: {e}")